In [ ]:
# 1. 필요한 패키지 설치
!pip install fastapi pyngrok uvicorn transformers accelerate nest_asyncio --quiet

In [ ]:
# 2. 모델 로딩
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import torch

model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 모델과 토크나이저 로드
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 3. 텍스트 생성 함수 (최적화 버전)
'''
def generate_text(prompt, max_length=512):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,               # 창의성 높임
        top_p=0.95,                   # nucleus sampling
        repetition_penalty=1.1,       # 반복 방지
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)
'''

In [ ]:
# 4. 정규표현식 파싱 함수 정의
import re

def parse_problems_from_text(text):
    pattern = r"Question\s*[:：]\s*(.*?)\s*Answer\s*[:：]\s*(.*?)(?=\nQuestion|\Z)"
    matches = re.findall(pattern, text, re.DOTALL)
    return [{"Question": q.strip(), "Answer": a.strip()} for q, a in matches]

In [ ]:
# 5. FastAPI app 설정
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline

app = FastAPI()
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

class ProblemRequest(BaseModel):
    question_type: str
    topic: str
    count: int

@app.post("/generate")
def generate_problems(req: ProblemRequest):
    prompt = f"""
    Generate {req.count} {req.question_type} questions about {req.topic}.

    Question: ...
    Answer: ...
    (repeat {req.count} times)
    """

    result = generator(prompt, max_length=400, do_sample=True, temperature=0.8, top_p=0.9, repetition_penalty=1)
    output = result[0]["generated_text"]
    print("\nGPT Output:\n", output)

    try:
        problems = parse_problems_from_text(output)
        if not problems:
            raise ValueError("No questions could be parsed.")
    except Exception as e:
        problems = [{"Question": output if isinstance(output, str) else str(output), "Answer": f"⚠️ Parsing error: {str(e)}"}]

    return {
        "results": problems,
        "raw_output": output
    }

In [ ]:
# 6. Run ngrok + uvicorn server (Colab only)
from pyngrok import ngrok
import nest_asyncio
import uvicorn

ngrok.set_auth_token("2v1Fi5CEzLumREBpheNMIIepRlM_7uLFbq5PGe81hmEZiAe9K")  # Replace with your ngrok token
ngrok.kill()  # Kill previous tunnels

public_url = ngrok.connect(3000)
print("🔗 Public URL:", public_url.public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=3000)


🔗 Public URL: https://9773-34-73-244-5.ngrok-free.app


INFO:     Started server process [584]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)


INFO:     155.230.85.158:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     155.230.85.158:0 - "GET /openapi.json HTTP/1.1" 200 OK


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-17' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run


GPT Output:
 
    Generate 2 Basic questions about Math.

    Question: ...
    Answer: ...
    (repeat 2 times)
        Question:...
        Answer:...
        (repeat 2 times)
            Question:...
            Answer:...

    Answer:...
    (repeat 2 times)
        Question:...
        Answer:...
        (repeat 2 times)
            Question:...
            Answer:...

    Question:...
    Answer:...
    (repeat 2 times)
        Question:...
        Answer:...
        (repeat 2 times)
            Question:...
            Answer:...

    Question:...
    Answer:...
    (repeat 2 times)
        Question:...
        Answer:...
        (repeat 2 times)
            Question:...
            Answer:...

A:

This is an implementation of the Fibonacci sequence. It uses an iterative implementation of the recurrence:

fib(n) = 1 + f(n - 1
INFO:     155.230.85.158:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [584]
